In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
# Coordinates for tornado disasters in Columbus, 1950-2000: http://www.usa.com/columbus-oh-natural-disasters-extremes.htm
df = pd.read_csv()

TypeError: parser_f() missing 1 required positional argument: 'filepath_or_buffer'